---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [158]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [159]:
dataframe=pd.DataFrame(df,columns=['text'])
dataframe.head()

,text
0,03/25/93 Total time of visit (in minutes):\n
1,6/18/85 Primary Care Doctor:\n
2,sshe plans to move as of 7/8/71 In-Home Servic...
3,7 on 9/27/75 Audit C Score Current:\n
4,2/6/96 sleep studyPain Treatment Pain Level (N...


In [209]:
type1_1=dataframe['text'].str.extractall(r'(\d{1,2})[/-](\d{1,2})[/-](\d{2})\b')
type1_2=dataframe['text'].str.extractall(r'(\d{1,2})[/-](\d{1,2})[/-](\d{4})\b')
type1=pd.concat([type1_1,type1_2])
type1.reset_index(inplace=True)
type1_index=type1['level_0']
#type1_index
type1.columns=['level_0','match','month','day','year']
type1['year']=type1['year'].apply(str)
type1['year']=type1['year'].apply(lambda x: '19'+x if len(x)<=2 else x)
type1

,level_0,match,month,day,year
0,0,0,03,25,1993
1,1,0,6,18,1985
2,2,0,7,8,1971
3,3,0,9,27,1975
4,4,0,2,6,1996
5,5,0,7,06,1979
6,6,0,5,18,1978
7,7,0,10,24,1989
8,8,0,3,7,1986
9,9,0,4,10,1971


In [210]:
import re
txt='6/2008; 12/2009; 2009; 2010'

In [211]:
re.findall(r'\d{,2}[/]?\d{4}',txt)

['6/2008', '12/2009', '2009', '2010']

In [212]:
type2=dataframe['text'].str.extractall(r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[-.]* )(\d{1,2})[?:, -]*(\d{4})')
type2.reset_index(inplace=True)
type2_index=type2['level_0']
type2_index_to_keep=[]
for index in type2_index:
    if index not in type1_index:
        type2_index_to_keep.append(index)
type2_to_keep=type2[type2['level_0'].isin(type2_index_to_keep)]
type2_to_keep.columns=['level_0','match','month','day','year']
type2_to_keep['month']=type2_to_keep.month.apply(lambda x: x[:3])
type2_to_keep['month']=pd.to_datetime(type2_to_keep.month, format='%b').dt.month
type2_to_keep

,level_0,match,month,day,year
0,194,0,4,11,1990
1,195,0,5,30,2001
2,196,0,2,18,1994
3,197,0,2,18,1981
4,198,0,10,11,2013
5,199,0,1,24,1986
6,200,0,7,26,1978
7,201,0,12,23,1999
8,202,0,5,15,1989
9,203,0,9,06,1995


In [213]:
type3=dataframe['text'].str.extractall(r'((?:\d{1,2} ))?((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[?:, -]* )(\d{4})')
type3.reset_index(inplace=True)
type3_index=type3['level_0']
type3_index_to_keep=[]
for index in type3_index:
    if (index not in type2_index_to_keep) and (index not in type1_index):
        type3_index_to_keep.append(index)
type3_to_keep=type3[type3['level_0'].isin(type3_index_to_keep)]
type3_to_keep.columns=['level_0','match','day','month','year']
type3_to_keep['day']=type3_to_keep['day'].fillna(1)
type3_to_keep['month']=type3_to_keep.month.apply(lambda x: x[:3])
type3_to_keep['month']=pd.to_datetime(type3_to_keep.month, format='%b').dt.month
type3_to_keep

,level_0,match,day,month,year
0,125,0,24,1,2001
1,126,0,10,9,2004
2,127,0,26,5,1982
3,128,0,28,6,2002
4,129,0,06,5,1972
5,130,0,25,10,1987
6,131,0,14,10,1996
7,132,0,30,11,2007
8,133,0,28,6,1994
9,134,0,14,1,1981


In [214]:
#type4=dataframe['text'].str.extractall(r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*)[ ](\d{,2}[a-z]+[,])[ ](\d{4})')
#type4=dataframe['text'].str.extractall(r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z] )((?:\d{1,2}(?:st|nd|rd|th), )\d{4})')
#type4.reset_index(inplace=True)
#type4.reset_index(inplace=True)
#type4

In [215]:
#type5=dataframe['text'].str.extractall(r'[^0-9][ ]((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*)[ ](\d{4})')
#type5=pd.DataFrame(type5)
#type5.reset_index(inplace=True)
#type5

In [216]:
type6=dataframe['text'].str.extractall(r'(\d{1,2})[/](\d{4})')
#type6=pd.DataFrame(type6)
type6.reset_index(inplace=True)
type6_index=type6['level_0']
type6_index_to_keep=[]
for index in type6_index:
    if (index not in type1_index) and (index not in type2_index_to_keep) and (index not in type3_index_to_keep):
        type6_index_to_keep.append(index)
type6_to_keep=type6[type6['level_0'].isin(type6_index_to_keep)]
type6_to_keep.columns=['level_0','match','month','year']
type6_to_keep['day']=1
type6_to_keep

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,level_0,match,month,year,day
25,343,0,6,1998,1
26,344,0,6,2005,1
27,345,0,10,1973,1
28,346,0,9,2005,1
29,347,0,03,1980,1
30,348,0,12,2005,1
31,349,0,5,1987,1
32,350,0,5,2004,1
33,351,0,8,1974,1
34,352,0,3,1986,1


In [217]:
type7_1=dataframe['text'].str.extractall(r'[a-z]?[^0-9](\d{4})[^0-9]')
type7_2=dataframe['text'].str.extractall(r'^(\d{4})[^0-9]')
type7=pd.concat([type7_1,type7_2])
type7.reset_index(inplace=True)
type7_index=type7['level_0']
type7_index_to_keep=[]
for index in type7_index:
    if (index not in type1_index) and (index not in type2_index_to_keep) and (index not in type3_index_to_keep) and (index not in type6_index_to_keep):
        type7_index_to_keep.append(index)
type7_to_keep=type7[type7['level_0'].isin(type7_index_to_keep)]
type7_to_keep.columns=['level_0','match','year']
type7_to_keep['day']=1
type7_to_keep['month']=1
type7_to_keep

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,level_0,match,year,day,month
356,455,0,1984,1,1
357,456,0,2000,1,1
358,457,0,2001,1,1
359,458,0,1982,1,1
360,459,0,1998,1,1
361,460,0,2012,1,1
362,461,0,1991,1,1
363,462,0,1988,1,1
364,463,0,2014,1,1
365,464,0,2016,1,1


In [243]:
all_date=pd.concat([type1,type2_to_keep,type3_to_keep,type6_to_keep,type7_to_keep])
all_date['date']=pd.to_datetime(all_date['day'].apply(str)+'/'+all_date['month'].apply(str)+'/'+all_date['year'].apply(str))
all_date=all_date.sort_values(by='date')
all_date

,day,level_0,match,month,year,date
67,18,84,0,5,1971,1971-05-18
2,8,2,0,7,1971,1971-08-07
9,10,9,0,4,1971,1971-10-04
44,11,53,0,7,1971,1971-11-07
24,12,28,0,9,1971,1971-12-09
375,1,474,0,1,1972,1972-01-01
28,13,153,0,1,1972,1972-01-13
13,26,13,0,1,1972,1972-01-26
46,04,171,0,10,1972,1972-04-10
77,13,98,0,5,1972,1972-05-13


In [244]:
s = type1.level_0.values.tolist()+type2.level_0.values.tolist()+type3_to_keep.level_0.values.tolist()+type6_to_keep.level_0.values.tolist()+type7_to_keep.level_0.values.tolist()
len(s)

500

In [246]:
orders=[]
for i in range(500):
    orders.append(i)
all_date['order']=orders
all_date['order']=all_date['order'].apply(lambda x: x+1)
all_date=all_date.sort_values(by='level_0').set_index('level_0')
all_date

,day,match,month,year,date,order
level_0,,,,,,
0,25,0,03,1993,1993-03-25,276
1,18,0,6,1985,1985-06-18,173
2,8,0,7,1971,1971-08-07,2
3,27,0,9,1975,1975-09-27,57
4,6,0,2,1996,1996-06-02,316
5,06,0,7,1979,1979-06-07,104
6,18,0,5,1978,1978-05-18,93
7,24,0,10,1989,1989-10-24,229
8,7,0,3,1986,1986-07-03,189


In [250]:
answer=pd.Series(all_date['order'],dtype='int')
answer

level_0
0      276
1      173
2        2
3       57
4      316
5      104
6       93
7      229
8      189
9        3
10     177
11      56
12     330
13       8
14     238
15     463
16     142
17      68
18     335
19      78
20     337
21     106
22     236
23      69
24     163
25     134
26     228
27      65
28       5
29     193
      ... 
470    143
471    340
472    447
473     46
474      6
475    489
476    215
477    280
478    270
479    311
480    478
481     27
482    231
483    294
484    388
485    196
486     16
487    256
488     71
489    165
490    419
491    440
492    178
493     82
494    373
495     97
496    409
497    431
498    403
499    107
Name: order, Length: 500, dtype: int64

In [257]:
#def date_sorter():
#    import pandas as pd
#    import numpy as np
#    doc = []
#    with open('dates.txt') as file:
#        for line in file:
#            doc.append(line)
#    df = pd.Series(doc)
#    
#    dataframe=pd.DataFrame(df,columns=['text'])
#    type1_1=dataframe['text'].str.extractall(r'(\d{1,2})[/-](\d{1,2})[/-](\d{2})\b')
#    type1_2=dataframe['text'].str.extractall(r'(\d{1,2})[/-](\d{1,2})[/-](\d{4})\b')
#    type1=pd.concat([type1_1,type1_2])
#    type1.reset_index(inplace=True)
#    type1_index=type1['level_0']
#    type1.columns=['level_0','match','month','day','year']
#    type1['year']=type1['year'].apply(str)
#    type1['year']=type1['year'].apply(lambda x: '19'+x if len(x)<=2 else x)
#    
#    type2=dataframe['text'].str.extractall(r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[-.]* )(\d{1,2})[?:, -]*(\d{4})')
#    type2.reset_index(inplace=True)
#    type2_index=type2['level_0']
#    type2_index_to_keep=[]
#    for index in type2_index:
#        if index not in type1_index:
#            type2_index_to_keep.append(index)
#    type2_to_keep=type2[type2['level_0'].isin(type2_index_to_keep)]
#    type2_to_keep.columns=['level_0','match','month','day','year']
#    type2_to_keep['month']=type2_to_keep.month.apply(lambda x: x[:3])
#    type2_to_keep['month']=pd.to_datetime(type2_to_keep.month, format='%b').dt.month
#    type2_to_keep
#    
#    type3=dataframe['text'].str.extractall(r'((?:\d{1,2} ))?((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[?:, -]* )(\d{4})')
#    type3.reset_index(inplace=True)
#    type3_index=type3['level_0']
#    type3_index_to_keep=[]
#    for index in type3_index:
#        if (index not in type2_index_to_keep) and (index not in type1_index):
#            type3_index_to_keep.append(index)
#    type3_to_keep=type3[type3['level_0'].isin(type3_index_to_keep)]
#    type3_to_keep.columns=['level_0','match','day','month','year']
#    type3_to_keep['day']=type3_to_keep['day'].fillna(1)
#    type3_to_keep['month']=type3_to_keep.month.apply(lambda x: x[:3])
#    type3_to_keep['month']=pd.to_datetime(type3_to_keep.month, format='%b').dt.month
#    
#    type6=dataframe['text'].str.extractall(r'(\d{1,2})[/](\d{4})')
#    type6.reset_index(inplace=True)
#    type6_index=type6['level_0']
#    type6_index_to_keep=[]
#    for index in type6_index:
#        if (index not in type1_index) and (index not in type2_index_to_keep) and (index not in type3_index_to_keep):
#            type6_index_to_keep.append(index)
#    type6_to_keep=type6[type6['level_0'].isin(type6_index_to_keep)]
#    type6_to_keep.columns=['level_0','match','month','year']
#    type6_to_keep['day']=1
#    
#    type7_1=dataframe['text'].str.extractall(r'[a-z]?[^0-9](\d{4})[^0-9]')
#    type7_2=dataframe['text'].str.extractall(r'^(\d{4})[^0-9]')
#    type7=pd.concat([type7_1,type7_2])
#    type7.reset_index(inplace=True)
#    type7_index=type7['level_0']
#    type7_index_to_keep=[]
#    for index in type7_index:
#        if (index not in type1_index) and (index not in type2_index_to_keep) and (index not in type3_index_to_keep) and (index not in type6_index_to_keep):
#            type7_index_to_keep.append(index)
#    type7_to_keep=type7[type7['level_0'].isin(type7_index_to_keep)]
#    type7_to_keep.columns=['level_0','match','year']
#    type7_to_keep['day']=1
#    type7_to_keep['month']=1
#    
#    all_date=pd.concat([type1,type2_to_keep,type3_to_keep,type6_to_keep,type7_to_keep])
#    all_date['date']=pd.to_datetime(all_date['day'].apply(str)+'/'+all_date['month'].apply(str)+'/'+all_date['year'].apply(str))
    #all_date=all_date.sort_values(by='date')
    
    #orders=[]
    #for i in range(500):
        #orders.append(i)
    #all_date['order']=orders
    #all_date['order']=all_date['order'].apply(lambda x: x+1)
#    all_date=all_date.sort_values(by='level_0').set_index('level_0')
#    date = all_date['date']
#    answer = pd.Series([i[0] for i in sorted(enumerate(date), key=lambda x:x[1])],np.arange(500))
    #answer=pd.Series(all_date['order'],dtype='int')
#    return answer
#date_sorter()

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers

0       84
1        2
2        9
3       53
4       28
5      474
6      153
7       13
8      171
9       98
10     129
11     225
12      31
13     111
14     191
15     486
16     335
17     415
18     323
19     405
20     422
21     375
22     380
23     345
24      57
25      36
26     481
27     436
28     299
29     402
      ... 
470    243
471    320
472    383
473    220
474    139
475     34
476    244
477    286
478    480
479    431
480    279
481    198
482    381
483    463
484    366
485    255
486    439
487    401
488    475
489    257
490    235
491    152
492    464
493    253
494    231
495    427
496    413
497    141
498    186
499    161
Length: 500, dtype: int64

In [ ]:
def date_sorter():
    import numpy as np
## (1) 04/20/2009; 04/20/09; 4/20/09; 4/3/09
    a1_1 =df.str.extractall(r'(\d{1,2})[/-](\d{1,2})[/-](\d{2})\b')
    a1_2 =df.str.extractall(r'(\d{1,2})[/-](\d{1,2})[/-](\d{4})\b')
    a1 = pd.concat([a1_1,a1_2])
    a1.reset_index(inplace=True)
    a1_index = a1['level_0']

## (2)Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009;
    a2 = df.str.extractall(r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[-.]* )((?:\d{1,2}[?:, -]*)\d{4})')
    a2.reset_index(inplace=True)
    a2_index = a2['level_0']
    
## (3) 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
## (5) Feb 2009; Sep 2009; Oct 2010
    a3 = df.str.extractall(r'((?:\d{1,2} ))?((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[?:, -]* )(\d{4})')
    a3.reset_index(inplace=True)
    a3_index = a3['level_0']

    ## (4) Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
#    a4 = df.str.extractall(r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z] )((?:\d{1,2}(?:st|nd|rd|th), )\d{4})')
#    a4.reset_index(inplace=True)
#    a4_index = a4['level_0']

    ## (6) 6/2008; 12/2009
    a6 = df.str.extractall(r'(\d{1,2})[/](\d{4})')
    a6.reset_index(inplace=True)
    a6_index = a6['level_0']
    save=[]
    for i in a6_index:
        if not(i in a1_index.values):
            save.append(i)
    save = np.asarray(save)
    a6 = a6[a6['level_0'].isin(save)]

    ## (7) 2009; 2010
    a7_1= df.str.extractall(r'[a-z]?[^0-9](\d{4})[^0-9]')
    a7_2 = df.str.extractall(r'^(\d{4})[^0-9]')
    a7 = pd.concat([a7_1,a7_2])
    a7.reset_index(inplace=True)

    a7_index = a7['level_0']
    save=[]
    for i in a7_index:
        if not((i in a2_index.values) | (i in a3_index.values) | (i in a6_index.values)):
            save.append(i)
    save = np.asarray(save)
    a7 = a7[a7['level_0'].isin(save)]
    
    s = a1.level_0.values.tolist()+a2.level_0.values.tolist()+a3.level_0.values.tolist()+a6.level_0.values.tolist()+a7.level_0.values.tolist()
    s = np.asarray(s)
#    print(diff(np.arange(500),s))
    
    a1.columns=['level_0','match','month','day','year']
    a1['year']=a1['year'].apply(str)
    a1['year']=a1['year'].apply(lambda x: '19'+x if len(x)<=2 else x)
   
    a2[1] = a2[1].apply(lambda x: x.replace(',',''))
    a2['day'] = a2[1].apply(lambda x:x.split(' ')[0])
    a2['year'] = a2[1].apply(lambda x:x.split(' ')[1])
    a2.columns=['level_0','match','month','day-year','day','year']
    a2.drop('day-year',axis=1,inplace=True) 
    
    a3.columns=['level_0','match','day','month','year']
    a3['day'] = a3['day'].replace(np.nan,-99)
    a3['day'] = a3['day'].apply(lambda x: 1 if int(x)==-99 else x)

    a3['month'] = a3.month.apply(lambda x: x[:3])
    a3['month'] = pd.to_datetime(a3.month, format='%b').dt.month
    
    a6.columns=['level_0','match','month','year']
    a6['day']=1
  
    a7.columns=['level_0','match','year']
    a7['day']=1
    a7['month']=1
   
    final = pd.concat([a1,a2,a3,a6,a7])
    final['date'] =pd.to_datetime(final['month'].apply(str)+'/'+final['day'].apply(str)+'/'+final['year'].apply(str))
    final = final.sort_values(by='level_0').set_index('level_0')

    myList = final['date']
    answer = pd.Series([i[0] for i in sorted(enumerate(myList), key=lambda x:x[1])],np.arange(500))
#    answer = pd.DataFrame([i[0] for i in sorted(enumerate(myList), key=lambda x:x[1])],np.arange(500)).reset_index()
#    answer.columns=['order','level_0']

#    answer= pd.DataFrame(final['date'].rank(ascending=1,method='first')).sort_values(by='date').reset_index()
#    answer.columns=['level_0','order']
#    answer['order']=answer['order']+1
    
#    final.reset_index(inplace=True)
#    real_final = pd.merge(final,answer,left_on='level_0',right_on='level_0',how='outer')
#    return real_final.order
    return answer